# Deploying a thai-embedder model (Snowflake/snowflake-arctic-embed-l-v2.0)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact your Cloud Pack for Data administrator and ask them for your account credentials

In [1]:
!pip install sentence-transformers==3.0.0
!pip install -U ibm-watson-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 75.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2


In [3]:
import numpy as np
import requests
from ibm_watson_machine_learning import APIClient

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url` and `api_key`.

In [4]:
# For on premise

# wml_credentials = {
#     "username": username,
#     "apikey": api_key,
#     "url": url,
#     "instance_id": 'openshift',
#     "version": '4.6'
# }

# For Saas
url='https://us-south.ml.cloud.ibm.com'
api_key='APIKEY'
wml_credentials = {
        "url": url,
        "apikey": api_key
    }

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.5/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [5]:
space_id = 'SPACEID'

You can use `list` method to print all existing spaces.

In [6]:
client.spaces.list(limit=10)

------------------------------------  --------------------------------------  ------------------------
ID                                    NAME                                    CREATED
c0c2c707-16bf-4620-929c-b0b5dae3116c  hugging-face-level1                     2024-04-13T08:44:07.135Z
e01b6ae7-3154-42ba-9fd3-ea298462286c  deploy                                  2024-02-28T09:28:04.456Z
4952f856-65b4-4df7-8523-08a012674e68  dep                                     2024-02-09T08:23:16.229Z
5fb9392c-787b-43a3-91e3-49c3b0355379  Tuning Space                            2024-02-07T05:27:24.684Z
f45fc607-d0b7-47fe-bdd7-65933e349238  deploy_gov_test2                        2023-12-27T10:15:15.265Z
a241367f-54e4-4fc8-b60a-d36ccb9e6bb1  deploy_gov_test                         2023-12-27T10:14:50.217Z
1975323a-9edf-4466-a381-6c104e5522f5  deploy_sepi2                            2023-12-27T10:13:42.684Z
5905937b-648b-4595-bd0a-0e330f8b099d  watsonx-governance-production           2023-12-27T0

,ID,NAME,CREATED
0,c0c2c707-16bf-4620-929c-b0b5dae3116c,hugging-face-level1,2024-04-13T08:44:07.135Z
1,e01b6ae7-3154-42ba-9fd3-ea298462286c,deploy,2024-02-28T09:28:04.456Z
2,4952f856-65b4-4df7-8523-08a012674e68,dep,2024-02-09T08:23:16.229Z
3,5fb9392c-787b-43a3-91e3-49c3b0355379,Tuning Space,2024-02-07T05:27:24.684Z
4,f45fc607-d0b7-47fe-bdd7-65933e349238,deploy_gov_test2,2023-12-27T10:15:15.265Z
5,a241367f-54e4-4fc8-b60a-d36ccb9e6bb1,deploy_gov_test,2023-12-27T10:14:50.217Z
6,1975323a-9edf-4466-a381-6c104e5522f5,deploy_sepi2,2023-12-27T10:13:42.684Z
7,5905937b-648b-4595-bd0a-0e330f8b099d,watsonx-governance-production,2023-12-27T08:43:23.572Z
8,db643a93-e681-45de-b4d0-118f262e4bff,watsonx-governance-incubation-practice,2023-12-27T08:23:47.787Z
9,7716c0a1-d194-453d-b96e-07c332b26df7,watsonx-governance-bootcamp,2023-12-27T07:36:24.395Z


To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [7]:
client.set.default_space(space_id)

'SUCCESS'

## Model Prep

In [8]:
%%writefile environment.yml
channels:
  - empty
  - nodefaults
dependencies:
- pip:
  - sentence-transformers==3.0.0

Writing environment.yml


In [9]:
def my_embedding_function():
    from sentence_transformers import SentenceTransformer
    # model_name = 'kornwtp/simcse-model-phayathaibert'  # Replace with the desired model name
    # Load the model
    model_name = 'Snowflake/snowflake-arctic-embed-l-v2.0'
    try:
        model = SentenceTransformer(model_name)
    except Exception as e:
        return {"error": str(e)}
    
    def score(payload):
        # we assume only one batch is sent
        sentences = payload['input_data'][0]['values'][0]
        try:
            embeddings = model.encode(sentences)
            return {
                'predictions': [
                    {
                        'fields': ['sentence', 'embedding'],
                        'values': [[sentence, embedding.tolist()] for sentence, embedding in zip(sentences, embeddings)]
                    }
                ]
            }
        except Exception as e:
            return {"error": str(e)}
    return score

# Example usage:
embedding_function = my_embedding_function()

# Example payload
payload = {
    'input_data': [
        {
            'values': [
                ["กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด", "กลุ่มเด็กชายกำลังเล่นฟุตบอลบนชายหาด"]
            ]
        }
    ]
}

result = embedding_function(payload)
print(result)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

{'predictions': [{'fields': ['sentence', 'embedding'], 'values': [['กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด', [-0.010231428779661655, 0.03986641392111778, -0.042735446244478226, -0.07801564782857895, -0.02775867097079754, -0.011078746989369392, 0.04653536528348923, 0.006247743498533964, -0.018073633313179016, -0.03230619058012962, 0.0013638574164360762, 0.003952351864427328, -0.006624063476920128, -0.008124599233269691, 0.03322526067495346, -0.012820265255868435, 0.08280769735574722, -0.04554062336683273, 0.07518007606267929, 0.014035808853805065, -0.06600766628980637, -0.017680874094367027, 0.016119062900543213, 0.02654307521879673, -0.019373562186956406, -0.020762771368026733, 0.021892499178647995, -0.01066155917942524, -0.022246651351451874, -0.02212711237370968, -0.0077729313634335995, 0.030249232426285744, 0.0379086434841156, -0.056836262345314026, -0.046900276094675064, 0.04048091918230057, -0.010090958327054977, 0.017412574961781502, -0.1286299228668213, -0.006412295158952475, 0.015991654

In [10]:
client.repository.list_functions()

------------------------------------  -------------------  ------------------------  ------  ----------  ----------------
GUID                                  NAME                 CREATED                   TYPE    SPEC_STATE  SPEC_REPLACEMENT
e8c5fc81-321f-4ecd-a4ec-cb2413a9b916  simcse embedder-GPU  2024-12-26T17:36:06.977Z  python
660402af-313b-4e5e-b3be-e4f82fd4fb55  simcse embedder-GPU  2024-12-26T17:34:38.340Z  python
4c21af19-0e57-4d00-910d-07fb34f95cc0  simcse embedder-v2   2024-12-11T08:51:12.968Z  python
2c52a08e-15ca-42a5-865d-a52677254455  simcse embedder-v2   2024-12-11T08:50:10.340Z  python
f1aeefdf-bce8-417b-bccb-356c6a040bfe  simcse embedder-v2   2024-12-11T08:47:30.165Z  python
7becde61-9fca-497d-a363-3656ea52b7b4  simcse embedder-v2   2024-12-11T08:45:56.607Z  python
------------------------------------  -------------------  ------------------------  ------  ----------  ----------------


,GUID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,e8c5fc81-321f-4ecd-a4ec-cb2413a9b916,simcse embedder-GPU,2024-12-26T17:36:06.977Z,python,,
1,660402af-313b-4e5e-b3be-e4f82fd4fb55,simcse embedder-GPU,2024-12-26T17:34:38.340Z,python,,
2,4c21af19-0e57-4d00-910d-07fb34f95cc0,simcse embedder-v2,2024-12-11T08:51:12.968Z,python,,
3,2c52a08e-15ca-42a5-865d-a52677254455,simcse embedder-v2,2024-12-11T08:50:10.340Z,python,,
4,f1aeefdf-bce8-417b-bccb-356c6a040bfe,simcse embedder-v2,2024-12-11T08:47:30.165Z,python,,
5,7becde61-9fca-497d-a363-3656ea52b7b4,simcse embedder-v2,2024-12-11T08:45:56.607Z,python,,


In [11]:
client.software_specifications.list()

----------------------------  ------------------------------------  ----  ------------  --------------------------
NAME                          ID                                    TYPE  STATE         REPLACEMENT
default_py3.6                 0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  retired       runtime-24.1-py3.11
autoai-ts_rt23.1-py3.10       01ce9391-1a79-5a33-94fb-2e134337f314  base  constricted   autoai-ts_rt24.1-py3.11
kernel-spark3.2-scala2.12     020d69ce-7ac1-5e68-ac1a-31189867356a  base  retired
pytorch-onnx_1.3-py3.7-edt    069ea134-3346-5748-b513-49120e15d288  base  retired
tensorflow_rt23.1-py3.10      079a91e0-245f-5269-8926-3c20b28f37dc  base  constricted   tensorflow_rt24.1-py3.11
scikit-learn_0.20-py3.6       09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  retired       runtime-24.1-py3.11
spark-mllib_3.0-scala_2.12    09f4cff0-90a7-5899-b9ed-1ef348aebdee  base  retired
pytorch-onnx_rt22.1-py3.9     0b848dd4-e681-5599-be41-b5f6fccc6471  base  retired       pytorch-on

,NAME,ID,TYPE,STATE,REPLACEMENT
0,default_py3.6,0062b8c9-8b7d-44a0-a9b9-46c416adcbd9,base,retired,runtime-24.1-py3.11
1,autoai-ts_rt23.1-py3.10,01ce9391-1a79-5a33-94fb-2e134337f314,base,constricted,autoai-ts_rt24.1-py3.11
2,kernel-spark3.2-scala2.12,020d69ce-7ac1-5e68-ac1a-31189867356a,base,retired,
3,pytorch-onnx_1.3-py3.7-edt,069ea134-3346-5748-b513-49120e15d288,base,retired,
4,tensorflow_rt23.1-py3.10,079a91e0-245f-5269-8926-3c20b28f37dc,base,constricted,tensorflow_rt24.1-py3.11
5,scikit-learn_0.20-py3.6,09c5a1d0-9c1e-4473-a344-eb7b665ff687,base,retired,runtime-24.1-py3.11
6,spark-mllib_3.0-scala_2.12,09f4cff0-90a7-5899-b9ed-1ef348aebdee,base,retired,
7,pytorch-onnx_rt22.1-py3.9,0b848dd4-e681-5599-be41-b5f6fccc6471,base,retired,pytorch-onnx_rt24.1-py3.11
8,ai-function_0.1-py3.6,0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda,base,retired,runtime-24.1-py3.11
9,watsonx-cfm-caikit-1.0,0cee3c55-472f-57b1-84bd-72f5d066dbe4,base,not_provided,


In [12]:
client.hardware_specifications.list() 

----------------------  ------------------------------------  --------------------------------------------------------------------------------------------------------------------------------------
NAME                    ID                                    DESCRIPTION
WXaaS-XL                0f174b00-46c7-5283-9db8-30bf2d429b49  watsonx.ai extra large
ML                      5a1f0e64-e420-55ce-bd7a-f6d00bb942cf  A hardware specification providing 4 CPU cores and 32 GiB of memory.
WXaaS-L                 5f56db8f-0e15-5a59-bd3b-1a72e28855ec  watsonx.ai large
WXaaS-S                 98da02d8-a815-5384-b4c0-2062515e2a16  watsonx.ai small
V100x2                  a02f3ab5-6964-4f06-a870-c7cc69187895  2 V100 GPU
L                       a6c4923b-b8e4-444c-9f43-8a7ec3020110  A hardware specification providing 8 CPU cores and 32 GiB of memory.
Default Spark           ac59d20a-9c7c-4504-a853-788ef35969da  A hardware specification for Spark with 1 CPU and 4 GiB of memory for master and worker n

,NAME,ID,DESCRIPTION
0,WXaaS-XL,0f174b00-46c7-5283-9db8-30bf2d429b49,watsonx.ai extra large
1,ML,5a1f0e64-e420-55ce-bd7a-f6d00bb942cf,A hardware specification providing 4 CPU cores...
2,WXaaS-L,5f56db8f-0e15-5a59-bd3b-1a72e28855ec,watsonx.ai large
3,WXaaS-S,98da02d8-a815-5384-b4c0-2062515e2a16,watsonx.ai small
4,V100x2,a02f3ab5-6964-4f06-a870-c7cc69187895,2 V100 GPU
5,L,a6c4923b-b8e4-444c-9f43-8a7ec3020110,A hardware specification providing 8 CPU cores...
6,Default Spark,ac59d20a-9c7c-4504-a853-788ef35969da,A hardware specification for Spark with 1 CPU ...
7,XXS,b128f957-581d-46d0-95b6-8af5cd5be580,A hardware specification providing one CPU cor...
8,M-Spark,b2232f7a-bfad-4822-9bce-6ba1af49217a,A hardware specification for Spark service wit...
9,M,c076e82c-b2a7-4d20-9c0f-1f0c2fdf5a24,A hardware specification providing 4 CPU cores...


In [18]:
meta_props = {
   client.package_extensions.ConfigurationMetaNames.NAME: "transformers",
   client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}
pkg_extn_details = client.package_extensions.store(meta_props, "./environment.yml")
pkg_extn_id = client.package_extensions.get_id(pkg_extn_details)


base_id = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
meta_props = {
   client.software_specifications.ConfigurationMetaNames.NAME: "default with sentence-transformer arctic",
   client.software_specifications.ConfigurationMetaNames.PACKAGE_EXTENSIONS: [{'guid': pkg_extn_id}],
   client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {'guid': base_id}
}
sw_spec_details = client.software_specifications.store(meta_props)
sw_spec_id = client.software_specifications.get_id(sw_spec_details)


# model_details = client.repository.store_model(
#     model=sample_saved_model_filename,
#     meta_props=metadata
# )

Creating package extensions
SUCCESS


In [19]:
function_props = {
    client.repository.FunctionMetaNames.NAME: 'snowflake-arctic-embed-l-v2.0',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sw_spec_id
}
function_details = client.repository.store_function(my_embedding_function, function_props)
function_id = client.repository.get_function_id(function_details)
print(function_id)

4e4962a0-e6e4-42d2-96a8-85d57d44a762


In [ ]:
hardware_spec_id = client.hardware_specifications.get_id_by_name('M')


deployment_props = {
    client.deployments.ConfigurationMetaNames.NAME: 'snowflake-arctic-embed-l-v2.0',
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id, 'num_nodes': 1}
}
deployment_details = client.deployments.create(function_id, deployment_props)
deployment_id = client.deployments.get_id(deployment_details)



#######################################################################################

Synchronous deployment creation for uid: '4e4962a0-e6e4-42d2-96a8-85d57d44a762' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
............

In [ ]:
payload = {
    'input_data': [
        {
            'values': [
                ["กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด", "กลุ่มเด็กชายกำลังเล่นฟุตบอลบนชายหาด"]
            ]
        }
    ]
}

client.deployments.score(deployment_id, payload)